In [21]:
##############################Kalman Filter operation#####################################
import numpy as np
import pybamm
from battery_data_analysis import BatteryPolynomialFit

pybamm.set_logging_level('INFO')

In [22]:
# Used to get the state of charge (SoC) from an open circuit voltage (OCV) reading,
# using a SoC(OCV) function calculated with an 8th order polynomial fit of the cell's quasi-OCV discharge at ~1/20C
# Inputs: the measured cell OCV from Arduino, coefficients form poly fit, OCV min and OCV max
# Returns: the state of charge SoC
def getSoCFromOCV(OCV, coefficients, OCV_min, OCV_max):
    OCV_used = OCV
    # ensures that only OCVs within the bounds of the polynomial fit are used
    if OCV_used < OCV_min:
        OCV_used = OCV_min
    if OCV_used > OCV_max:
        OCV_used = OCV_max
    # calculates the state of charge from the OCV
    SoC = 0
    i = 0
    coefficients_len = len(coefficients)
    while coefficients_len - i >= 0:
        SoC += coefficients[i] * OCV_used ** (coefficients_len - i)
        i += 1
    # returns the state of charge as a function of measured open circuit voltage (OCV)
    return SoC

# Initializes the state matrix for the Kalman Filter
# Inputs: the initial measured state of charge SoC and current I
# Returns: the state matrix to be used in the Kalman Filter
def initStateVariable(SoC, I):
    return np.array([SoC], [I])


# Updates the Kalman Filter's transformation matrix with the appropriate time interval
# Inputs: the measured time interval between operations
# Returns: the correct transformation matrix to be used in Kalman Filter operations
def updateTransformationMatrix(deltaT):
    totalCoulombs = 10800
    return np.array([[1, -1 * (deltaT / totalCoulombs) * 100], [0, 1]])


In [31]:
class PybammBattery:

    def __init__(self, env, capacity, soc=0, Vmin=2.8, Vmax=4.5):
        self.env = env
        self.capacity = capacity
        self.soc = soc
        
        # load solver
        self.solver = pybamm.CasadiSolver()
        
        # load model
        self.model = pybamm.lithium_ion.SPMe()
        
        # load parameter values and process model and geometry
        self.parameter_values = pybamm.ParameterValues('Marquis2019')
        
        # Create the BatteryPolynomialFit and get coefficients list
        self.battery_polynomial_fit = BatteryPolynomialFit(self.model.new_copy(), self.parameter_values.copy())
        
        # Set the step_solution to the last solution
        # self.step_solution = self.battery_polynomial_fit.last_solution
        hujj = self.battery_polynomial_fit.last_solution
        
        # Set initial voltage and initial SOC
#         Vmax = self.battery_polynomial_fit.Vmax
#         Vmin = self.battery_polynomial_fit.Vmin
#         coefficients_list = self.battery_polynomial_fit.coefficients_list
#         self.initial_voltage = self.step_solution['Terminal voltage [V]'].entries[-1]
#         self.initial_SOC = getSoCFromOCV(self.initial_voltage, coefficients_list, Vmin, Vmax)
#         self.initial_current = self.parameter_values['Current function [A]']
        
        geometry = self.model.default_geometry
        self.parameter_values['Current function [A]'] = "[input]"
        self.parameter_values.process_model(self.model)
        self.parameter_values.process_geometry(geometry)
        
        # set mesh
        mesh = pybamm.Mesh(geometry, self.model.default_submesh_types, self.model.default_var_pts)

        # discretise model
        disc = pybamm.Discretisation(mesh, self.model.default_spatial_methods)
        disc.process_model(self.model)
            
        
    def update(self, current):
        if current == 0:
            return 0
        
        if self.step_solution is not None:
            print('condition final time: ', not (
            self.step_solution.termination == "final time"
            or "[experiment]" in self.step_solution.termination
        ))
        
        input_parameters= {}
        input_parameters['Current function [A]'] = current
        self.step_solution = self.solver.step(self.step_solution, self.model, dt=60, npts=100, inputs=input_parameters)
        
        self.calculate_soc()
        
        
    def calculate_soc(self):
        print('negative electrode: ', self.step_solution['Negative electrode SOC'].data[-1])
        print('positive electrode: ', self.step_solution['Positive electrode SOC'].data[-1])
        print('Discharge capacity: ', self.step_solution['Discharge capacity [A.h]'].data[-1])
        
        self.neg_soc.append(self.step_solution['Negative electrode SOC'].data[-1])
        self.pos_soc.append(self.step_solution['Positive electrode SOC'].data[-1])

In [32]:
battery = PybammBattery(None, 5)

2022-05-04 17:47:45,275 - [INFO] base_battery_model.build_model(845): Start building Single Particle Model with electrolyte
2022-05-04 17:47:45,341 - [INFO] base_battery_model.build_model(865): Finish building Single Particle Model with electrolyte
2022-05-04 17:47:45,365 - [INFO] parameter_values.process_model(371): Start setting parameters for Single Particle Model with electrolyte
2022-05-04 17:47:45,465 - [INFO] parameter_values.process_model(474): Finish setting parameters for Single Particle Model with electrolyte
2022-05-04 17:47:45,466 - [INFO] discretisation.process_model(137): Start discretising Single Particle Model with electrolyte
2022-05-04 17:47:45,716 - [INFO] discretisation.process_model(254): Finish discretising Single Particle Model with electrolyte
2022-05-04 17:47:45,717 - [INFO] parameter_values.process_model(371): Start setting parameters for Single Particle Model with electrolyte
2022-05-04 17:47:45,821 - [INFO] parameter_values.process_model(474): Finish settin


FITTING RESULTS
---------------
Optimal Order Fit: 8
Chi-Squared Value: 2245.8965074292073
Coefficients:
	x^8: -378616.63867396466
	x^7: 10926007.548441665
	x^6: -137754589.37971863
	x^5: 991092475.1377708
	x^4: -4450423410.452015
	x^3: 12772196032.737785
	x^2: -22877346952.19099
	x^1: 23383139885.25651
	x^0: -10441745311.246473


AttributeError: 'BatteryPolynomialFit' object has no attribute 'last_solution'